In [ ]:
from scripts.qasrl_gs_utils import combine_files_tag_with_sentences

combine_files_tag_with_sentences("qasrl_gs/data/gold/wikinews.dev.gold.csv", "qasrl_gs/data/sentences/wikinews.dev.full.csv", "qasrl_gs/data/gold/wikinews.dev.combined.csv")
combine_files_tag_with_sentences("qasrl_gs/data/gold/wikinews.test.gold.csv", "qasrl_gs/data/sentences/wikinews.test.full.csv", "qasrl_gs/data/gold/wikinews.test.combined.csv")

# Run one time on new server

In [ ]:
!pip install -r requirements.txt
!python -m spacy download en_core_web_sm

# Run on every notebook restart

In [ ]:
# Imports

from run_summarization import main
import os
import sys
import json

# Params

qasrl_2015_params = ['--dataset_name', 'qa_srl']
qasrl_2020_params = [
    "--train_file", "qasrl_gs/data/gold/wikinews.dev.combined.csv",
    "--validation_file", "qasrl_gs/data/gold/wikinews.dev.combined.csv",
    "--test_file", "qasrl_gs/data/gold/wikinews.test.combined.csv",
    "--text_column", "sentence", 
    "--summary_column", "answer"
]

t5_small_model_params = [
    '--model_name_or_path', 't5-small',
    '--source_prefix', 'summarize: '
]
bart_model_params = [
    '--model_name_or_path', 'facebook/bart-base'
]

# Other

run = None  # wandb run

# Run once

In [ ]:
# !python run_summarization.py --model_name_or_path t5-small --do_train --do_eval --dataset_name qa_srl --source_prefix "summarize: " --output_dir $TMPDIR/tst-summarization --per_device_train_batch_size=4 --per_device_eval_batch_size=4 --num_train_epochs 3.0 --overwrite_output_dir
sys.argv = [
    'run_summarization.py',
    '--do_train',
    '--do_eval',
    '--output_dir', f'{os.environ["TMPDIR"]}/tst-summarization',
    '--per_device_train_batch_size', '4',
    '--per_device_eval_batch_size', '4',
    '--logging_steps', '100',
    '--num_train_epochs', '3.0',
    '--overwrite_output_dir',
    '--report_to', 'wandb'    
]
sys.argv.extend(t5_small_model_params)
sys.argv.extend(qasrl_2020_params)

_, run = main()

In [ ]:
# !python run_summarization.py --model_name_or_path $TMPDIR/tst-summarization --do_predict --dataset_name qa_srl --output_dir $TMPDIR/tst-summarization --source_prefix "summarize: " --predict_with_generate
sys.argv = [
    'run_summarization.py',
    '--model_name_or_path', f'{os.environ["TMPDIR"]}/tst-summarization',
    '--do_predict',
    '--output_dir', f'{os.environ["TMPDIR"]}/tst-summarization',
    '--source_prefix', 'summarize: ',
    '--predict_with_generate',
    '--eval_accumulation_steps', '10',  # Necessary to avoid OOM where all predictions are kept on one GPU    
    '--report_to', 'wandb',
    '--wandb_run_name', run.name if run else None
]
sys.argv.extend(t5_small_model_params)
sys.argv.extend(qasrl_2020_params)


main()

In [ ]:
with open ("/home/nlp/hirsche5/tmp/tst-summarization/generated_predictions.json") as f:
    predictions = json.loads(f.read())
list(zip(predictions['inputs'], predictions['labels'], predictions['predictions']))[:10]

In [ ]:
# !python run_summarization.py --model_name_or_path $TMPDIR/tst-summarization --do_predict_based_on_predictions_file --dataset_name qa_srl --output_dir $TMPDIR/tst-summarization --source_prefix "summarize: "
sys.argv = [
    'run_summarization.py',
    '--model_name_or_path', f'{os.environ["TMPDIR"]}/tst-summarization',
    '--do_predict_based_on_predictions_file',
    '--dataset_name', 'qa_srl',
    '--output_dir', f'{os.environ["TMPDIR"]}/tst-summarization',
    '--source_prefix', 'summarize: ',
    '--report_to', 'wandb',
    '--wandb_run_name', run.name    
]
main()

# Run experiments

In [ ]:

batch_size = 4
model_name_or_path = 't5-small'

# for batch_size in [4, 16]:
# for model in ['t5-small', 't5-large']:
for preprocess_output_func in ['all', 'first_two_question_answer']:
    sys.argv = [
        'run_summarization.py',
        '--model_name_or_path', model_name_or_path,
        '--do_train',
        '--do_eval',
        '--do_predict',
        '--predict_with_generate',
        '--do_predict_based_on_predictions_file',
        '--dataset_name', 'qa_srl',
        '--source_prefix', 'summarize: ',
        '--output_dir', f'{os.environ["TMPDIR"]}/tst-summarization',
        '--per_device_train_batch_size', str(batch_size),
        '--per_device_eval_batch_size', str(batch_size),
        '--num_train_epochs', '3.0',
        '--overwrite_output_dir',
        '--eval_accumulation_steps', '10',  # Necessary to avoid OOM where all predictions are kept on one GPU
        '--report_to', 'wandb'    
    ]

    main()

# Debug mode

In [ ]:
# !python run_summarization.py --model_name_or_path $TMPDIR/tst-summarization --do_predict --dataset_name qa_srl --output_dir $TMPDIR/tst-summarization --source_prefix "summarize: " --predict_with_generate --debug_mode
sys.argv = [
    'run_summarization.py',
    '--model_name_or_path', f'{os.environ["TMPDIR"]}/tst-summarization',
    '--do_predict',
    '--dataset_name', 'qa_srl',
    '--output_dir', f'{os.environ["TMPDIR"]}/tst-summarization',
    '--source_prefix', 'summarize: ',
    '--predict_with_generate',
    '--eval_accumulation_steps', '10',  # Necessary to avoid OOM where all predictions are kept on one GPU        
    '--debug_mode'
]
main()

In [ ]:
with open ("/home/nlp/hirsche5/tmp/tst-summarization/generated_predictions.json") as f:
    predictions = json.loads(f.read())
list(zip(predictions['inputs'], predictions['labels'], predictions['predictions']))[0]

In [ ]:
# !python run_summarization.py --model_name_or_path $TMPDIR/tst-summarization --do_predict_based_on_predictions_file --dataset_name qa_srl --output_dir $TMPDIR/tst-summarization --source_prefix "summarize: " --debug_mode --report_to "wandb"
sys.argv = [
    'run_summarization.py',
    '--model_name_or_path', f'{os.environ["TMPDIR"]}/tst-summarization',
    '--do_predict_based_on_predictions_file',
    '--dataset_name', 'qa_srl',
    '--output_dir', f'{os.environ["TMPDIR"]}/tst-summarization',
    '--source_prefix', 'summarize: ',
    '--debug_mode'
]
main()